# This notebook demonstrates how to generate vectorizations from source TIFF files

In [1]:
%ls /Users/awatters/misc/LisaBrown/movies

210501_NANOG_Gata6_with_protein_levels.mat
Gata6Nanog1.json
MakeTracksForAaronFromHaydensGraph.py
OutputGraphForPython.m
Stardist3D_klbOut_Cam_Long_00006.tif
Stardist3D_klbOut_Cam_Long_00007.tif
exploring.ipynb
fwdcontiguousmoviessomegapsstill.zip
test.json


In [2]:
# Load the arrays from the TIFF files...

file6 = "/Users/awatters/misc/LisaBrown/movies/Stardist3D_klbOut_Cam_Long_00006.tif"
file7 = "/Users/awatters/misc/LisaBrown/movies/Stardist3D_klbOut_Cam_Long_00007.tif"

In [3]:
from mouse_embryo_labeller.tools import load_tiff_array

In [4]:
labels6 = load_tiff_array(file6)

In [5]:
labels6.shape

(64, 800, 800)

In [6]:
labels7 = load_tiff_array(file7)
labels7.shape

(64, 800, 800)

In [7]:
# Load the JSON dump of the label correspondence

file_name = "Gata6Nanog1.json"
import json
json_graph = json.load(open(file_name))

In [8]:
from mouse_embryo_labeller.vectorizor import make_tracks_from_haydens_json_graph

timestamp_mapping = make_tracks_from_haydens_json_graph(json_graph)

In [9]:
# Get the mappings of labels to tracks for the label arrays

ts6m = timestamp_mapping[6]
ts6m


{5: 1, 3: 2, 2: 3, 6: 4, 4: 5, 7: 6, 8: 7, 9: 8, 1: 9}

In [10]:
ts7m = timestamp_mapping[7]
ts7m

{5: 1, 1: 2, 3: 3, 6: 4, 9: 5, 7: 6, 2: 7, 8: 8, 4: 9}

In [11]:
# generate the vectors for the full size arrays, this may take a few seconds...

from mouse_embryo_labeller.vectorizor import get_track_vector_field

vector_field = get_track_vector_field(
    old_label_array=labels6, 
    old_labels_to_tracks=ts6m,
    new_label_array=labels7,
    new_labels_to_tracks=ts7m,
    di = (10, 0, 0),  #  xyz offset between A[i,j,k] and A[i+1,j,k]
    dj = (0, 1, 0),  #  xyz offset between A[i,j,k] and A[i,j+1,k]
    dk = (0, 0, 1),  #  xyz offset between A[i,j,k] and A[i,j,k+1]
    )
vector_field.shape

(64, 800, 800, 3)

# Viewing a subsampled version of the arrays for testing

In [12]:
# looking at subsampled data
import numpy as np

jkstride = 20
istride = 2
ss6 = labels6[::istride, ::jkstride, ::jkstride]
ss6.shape, len(np.nonzero(ss6)[0])

((32, 40, 40), 213)

In [13]:
ss7 = labels7[::istride, ::jkstride, ::jkstride]
ss7.shape, len(np.nonzero(ss7)[0])

((32, 40, 40), 229)

In [14]:
from mouse_embryo_labeller.vectorizor import unify_tracks

# Generate the arrays of treck numbers from the arrays of label numbers

(tracks6, tracks7) = unify_tracks(
    A=ss6,  # label array
    A_label_2_track=ts6m,   # mapping of labels in A to track numbers
    B=ss7,  # label array
    B_label_2_track=ts7m,   # mapping of labels in B to track numbers
)


In [15]:
# generate vectors for the subsampled arrays

from mouse_embryo_labeller.vectorizor import VectorMaker

V = VectorMaker(tracks6, tracks7)
V.scaled_vectors.shape

(32, 40, 40, 3)

In [ ]:
# view the vectorizor widget

W = V.widget()